Visualize LHD meshes with plotly
===

In [ ]:
import os
import numpy as np
from stl import mesh
import plotly.graph_objects as go

CHERAB_ROOT = os.getcwd().split("docs")[0]
STL_DIR = os.path.join(CHERAB_ROOT, "cherab/lhd/machine/geometry/data/STLfiles")


Define local function to convert stl to maeh3D format for plotly
---

In [ ]:
def stl2mesh3d(stl_mesh):
    """Extracts the unique vertices and the lists I, J, K to define a Plotly mesh3d

    Parameters
    ----------
    stl_mesh : :obj:`stl.base.BaseMesh`
        loaded by numpy-stl module from a stl file

    Returns
    -------
    numpy.ndarray
        2D-array of vertices
    numpy.ndarray
        A vector of vertex indices, representing the "first" vertex of triangle
    numpy.ndarray
        A vector of vertex indices, representing the "second" vertex of triangle
    numpy.ndarray
        A vector of vertex indices, representing the "third" vertex of triangle
    """
    p, q, r = stl_mesh.vectors.shape  # (p, 3, 3)
    # the array stl_mesh.vectors.reshape(p * q, r) can contain multiple copies of the same vertex;
    # extract unique vertices from all mesh triangles
    vertices, ixr = np.unique(stl_mesh.vectors.reshape(p * q, r), return_inverse=True, axis=0)
    I = np.take(ixr, [3 * k for k in range(p)])
    J = np.take(ixr, [3 * k + 1 for k in range(p)])
    K = np.take(ixr, [3 * k + 2 for k in range(p)])
    return vertices, I, J, K


Vessel Wall + plasma
---

In [ ]:
# === Load Vessel =============================================================
vessel = mesh.Mesh.from_file(os.path.join(STL_DIR, "vessel.stl"))
vertices, I, J, K = stl2mesh3d(vessel)
x, y, z = vertices.T

# set figure
fig = go.Figure()

vessel_mesh = go.Mesh3d(
    x=x,
    y=y,
    z=z,
    i=I,
    j=J,
    k=K,
    flatshading=True,
    colorscale=[[0, "#e5dee5"], [1, "#e5dee5"]],
    opacity=0.5,
    intensity=z,
    name="Vessel",
    showscale=False,
    hoverinfo="skip",
    lighting=dict(
        ambient=0.18, diffuse=1, fresnel=0.1, specular=0, roughness=0.1, facenormalsepsilon=0
    ),
    lightposition=dict(x=3000, y=3000, z=10000),
)

fig.add_trace(vessel_mesh)

# === Load Plasma =============================================================
plasma = mesh.Mesh.from_file(os.path.join(STL_DIR, "plasma.stl"))
vertices, I, J, K = stl2mesh3d(plasma)
x, y, z = vertices.T

plasma_mesh = go.Mesh3d(
    x=x,
    y=y,
    z=z,
    i=I,
    j=J,
    k=K,
    flatshading=True,
    color="#FF00FF",
    opacity=0.1,
    # intensity=z,
    name="Plasma",
    showscale=False,
    hoverinfo="skip",
)

fig.add_trace(plasma_mesh)

# === update figure layout ====================================================
fig.update_scenes(
    xaxis_visible=False,
    yaxis_visible=False,
    zaxis_visible=False,
)
camera = dict(up=dict(x=0, y=0, z=1), center=dict(x=0, y=0, z=0), eye=dict(x=1.0, y=1.0, z=1.0))
fig.update_layout(
    paper_bgcolor="rgb(1,1,1)",
    title_text="Vessel Wall + Plasma",
    title_x=0.5,
    font_color="white",
    width=700,
    height=600,
    scene_camera=camera,
    margin=dict(r=10, l=10, b=10, t=40),
)

# save as html
# fig.write_html(os.path.join(STL_DIR, "vessel.html"))

fig.show()
